In [67]:
import boto3

region = boto3.Session().region_name
session = boto3.session.Session()

ec2 = boto3.Session().client(service_name="ec2", region_name=region)
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [68]:
import sagemaker
import time
from time import gmtime, strftime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})

iam = boto3.client("iam", config=config)

In [69]:
role_name = role.split("/")[-1]

print("Role name: {}".format(role_name))

Role name: LabRole


In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install --disable-pip-version-check -q PyAthena==2.1.0
! pip install descartes
! pip install geopandas
!pip install wordcloud
from pyathena import connect
import geopandas as gpd
from tqdm import tqdm  
from geopandas import GeoDataFrame, points_from_xy
from wordcloud import WordCloud

In [71]:
sess = sagemaker.Session()
bucket = '{}'.format(bucket)
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
bucket

'sagemaker-us-east-1-705927414280'

In [72]:
# Assign database name
database_name = "ads508"

In [73]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [74]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [75]:
# Create new database 'ads508'
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
create_db = pd.read_sql(statement, conn)
create_db

""


In [76]:
# Verify database creation
q = "SHOW DATABASES"
db_show = pd.read_sql(q, conn)
db_show

,database_name
0,ads508
1,default
2,dsoaws


In [77]:
# Set Athena parameters
database_name = "ads508"
model_table_name_csv = "modeling"
model_s3_path = "s3://{}/modeling_data".format(bucket)
print(model_s3_path)

s3://sagemaker-us-east-1-705927414280/modeling_data


In [78]:
df = pd.read_csv("{}/data_for_modeling.csv".format(model_s3_path))
df = df.drop(columns=['Unnamed: 0'])
df.head()

,latitude,longitude,n_guns_involved,target_class,group_Democrat,suspect_age,ohe_drug,ohe_officer,ohe_gang,ohe_accident,...,suspect_age_group_Adult,suspect_age_group_Senior,region_East South Central,region_Middle Atlantic,region_Mountain,region_New England,region_Pacific,region_South Atlantic,region_West North Central,region_West South Central
0,40.3467,-79.8559,1.0,1,1,Adult 18+,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,40.4555,-79.8970,1.0,1,1,NaN,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,33.9090,-118.3330,1.0,1,1,NaN,0,0,1,1,...,0,0,0,0,0,0,1,0,0,0
3,33.8447,-118.3070,1.0,1,1,Adult 18+,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,33.9454,-118.3990,1.0,1,1,NaN,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [79]:
# statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
#  latitude double,
#  longitude double,
#  n_guns_involved double,
#  target_class int,
#  group_Democrat int,
#  suspect_age string,
#  ohe_drug int,
#  ohe_officer int,
#  ohe_gang int,
#  ohe_accident int,
#  ohe_murder int,
#  ohe_suicide int,
#  ohe_arrest int,
#  ohe_brandishing int,
#  ohe_felon int,
#  ohe_drive int,
#  ohe_home_invasion int,
#  ohe_stolen int,
#  ohe_misc int,
#  ohe_drugs int,
#  ohe_car_jacking int,
#  ohe_defensive int,
#  ohe_robbery int,
#  ohe_family int,
#  ohe_institution int,
#  ohe_child int,
#  ohe_mass int,
#  ohe_domestic int,
#  suspect_age_group_Teen int,
#  suspect_age_group_Young_Adult int,
#  suspect_age_group_Mid-Adult int,
#  suspect_age_group_Adult int,
#  suspect_age_group_Senior int,
#  region_East_South_Central int,
#  region_Middle_Atlantic int,
#  region_Mountain int,
#  region_New_England int,
#  region_Pacific int,
#  region_South_Atlantic int,
#  region_West_North_Central int,
#  region_West_South_Central int
 
# ) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
# TBLPROPERTIES ('skip.header.line.count'='1')""".format(
#     database_name, model_table_name_csv, model_s3_path
# )

# create_table = pd.read_sql(statement, conn)
# create_table

Displaying the previous count of the target class variable. 
We find that the class of 1 - which indicates someone was either injured
or killed - is the majority class.

In [80]:
# count the number of 0s and 1s in the 'outcome' column
value_counts = df['target_class'].value_counts()

# print the results
print(value_counts)

1    131557
0     88789
Name: target_class, dtype: int64


# Undersampling the majority target_class of '1' 

In [81]:
df_grouped_by = df.groupby(["target_class"])
df_balanced = df_grouped_by.apply(
    lambda x: x.sample(df_grouped_by.size().min())\
    .reset_index(drop=True)
)

In [82]:
# showing the balanced value counts of the newly created dataframe
value_counts_balanced = df_balanced['target_class'].value_counts()

# print the results
print(value_counts_balanced)

0    88789
1    88789
Name: target_class, dtype: int64


In [83]:
from sklearn.model_selection import train_test_split

# Split all data into 90% train and 10% holdout
df_train, df_holdout = train_test_split(
        df_balanced,
        test_size=0.10,
    stratify=df_balanced['target_class'])

# Split holdout data into 50% validation and 50% test
df_validation, df_test = train_test_split(
        df_holdout,
        test_size=0.50,
        stratify=df_holdout['target_class'])